In [56]:
import pandas as pd 
import os 
import re 
import time
from datetime import datetime
import sqlite3
DATABASE = r"Z:\跨部門\共用資料夾\C. 業務部\詢價統計DB\QUOTATION_DATABASE.db"
chosen_column = [0, 1, 2, 3, 4, 5 ,18, 28, 29, 36, 38, 39, 44, 45, 46, 47]
COLUMNS = ["S_C", "ITEM", "SPEC", "PRODUCT_CODE", "UNIT", "ORDER_QUANTITY", "KG_Mpcs", "ORDER_NUMBER",
              "CUSTOMER_CODE", "ORDER_DATE", "QUANTITY_PER_CARTON",
           "QUANTITY_PER_BOX",
              "MERCHANDISE", "NOTE", "INNER_BOX_SPEC", "OUTER_BOX_SPEC"]
RAW = pd.read_csv("2024_March_order.csv", usecols = chosen_column)
RAW = RAW.astype(str)
RAW

,內部S/C NO.,項次,產品規格,客戶產品代號,計量單位,訂單量,千支重,客戶訂單號碼,下單客戶,訂單日期,每箱數量,每盒數量,產品說明,項次包裝備註,內盒規格,外箱規格
0,24030032,1,M3.9X30,DWC3930PR,M,"1,140.000",1.83,4500337986,D04001,2024/3/28,95.0,0.0,"1022A, 乾牆螺絲 十字#2 喇叭頭, 粗牙, 滲碳S.H, (酸洗)灰磷 48HR, ...","樣品: 150PCS (50/包,共3包)",nan,重包裝牛皮箱750*570*420mm(圍身5層)
1,24030031,1,"#6X1""",0280059,LBS,"7,200.000",1.35,322059,D09200,2024/3/28,25.0,0.0,"1022A, 乾牆螺絲 十字#2 喇叭頭, 雙牙單出(SS), 滲碳S.H, (酸洗)灰磷 ...",nan,nan,D092 (PK112+MF506)標準L系列灰色上蓋(PMS421-107282)+塑膠桶...
2,24030031,2,"#6X1-1/4""",0283072,M,"3,840.000",1.57,322059,D09200,2024/3/28,8.0,0.0,"1022A, 乾牆螺絲 十字#2 喇叭頭, 雙牙單出(SS), 滲碳S.H, (酸洗)灰磷 ...",nan,nan,D092 (PK305-500)PRO-FIT系列散裝外箱257x257x171MM
3,24030031,3,"#6X1-5/8""",0283102,M,"1,200.000",1.98,322059,D09200,2024/3/28,5.0,0.0,"1022A, 乾牆螺絲 十字#2 喇叭頭, 雙牙單出(SS), 滲碳S.H, (酸洗)灰磷 ...",nan,nan,D092 (PK305-500)PRO-FIT系列散裝外箱257x257x171MM
4,24030031,4,"#6X1-1/4""",0287072,M,"3,840.000",1.57,322059,D09200,2024/3/28,8.0,0.0,"1022A, 乾牆螺絲 十字#2 喇叭頭, 粗牙, 滲碳S.H, 灰磷48HR, 48小時無...",nan,nan,D092 (PK305-500)PRO-FIT系列散裝外箱257x257x171MM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,24030002,18,M3.9X22,89153.035.0039.022,M,732.000,1.16,3810439-457,C01900,2024/3/1,1.0,0.0,"1022A, 乾牆螺絲 十字#2 小頭 60度 4線, 高低牙尖尾, 滲碳S.H, (酸洗)...",nan,nan,C019 907-ARNB15自動彩盒-90x90x150MM
306,24030002,19,M3.9X30,89153.035.0039.030,M,258.000,1.48,3810439-457,C01900,2024/3/1,1.0,0.0,"1022A, 乾牆螺絲 十字#2 小頭 60度 4線, 高低牙尖尾, 滲碳S.H, (酸洗)...",nan,nan,C019 907-ARNB21自動彩盒-90x90x210MM
307,24030002,20,M3.9X35,89153.035.0039.035,M,516.000,1.69,3810439-457,C01900,2024/3/1,1.0,0.0,"1022A, 乾牆螺絲 十字#2 小頭 60度 4線, 高低牙尖尾, 滲碳S.H, (酸洗)...",nan,nan,C019 907-ARNB21自動彩盒-90x90x210MM
308,24030002,21,M3.9X19,89153.037.0039.019,M,366.000,1.03,3810439-457,C01900,2024/3/1,1.0,0.0,"1022A, 乾牆螺絲 十字#2 小頭 60度 4線, 高低牙尖尾, 滲碳S.H, (酸洗)...",nan,nan,C019 BAU!ES 908-ARNK15自動彩盒-90x90x150MM


In [53]:
def CHECK_FILE(DATABASE, TABLE):
    with sqlite3.connect(DATABASE) as connection :
        cursor = connection.cursor()
        # Replace 'your_table_name' and 'your_column_name' with the actual table and column names
        column_name = "S_C"
        # Check if the value already exists in the specified column
        query = f"SELECT {column_name} FROM {TABLE};"
        #existing_rows = cursor.execute(query).fetchall()
        df = pd.read_sql_query(query, connection)
    return df

In [55]:
def ORDER_INSERT(DATAFRAME, TABLENAME, DBNAME, VERNIER):
    ORDER_SC_LIST = CHECK_FILE(DATABASE, "ORDER_SUMMARY" )
    
    for quote_item in DATAFRAME.index:
        ROW_INSERT = DATAFRAME.loc[quote_item,:].values.flatten().tolist()
        if ROW_INSERT[0] not in ORDER_SC_LIST["S_C"].values: # 檢驗此單是否已被記錄過
            query = f"INSERT INTO {TABLENAME} ({', '.join(COLUMNS)}) VALUES ({', '.join(['?']*len(ROW_INSERT))});"
            VERNIER.execute(query, ROW_INSERT)
            time.sleep(0.1)
        else:
            pass

with sqlite3.connect(DATABASE) as connection : 
    cursor = connection.cursor()
    ORDER_INSERT(RAW, "ORDER_SUMMARY", DATABASE, cursor)

In [75]:
def FIND_QUOTE(product_name, quote_date):
    with sqlite3.connect(DATABASE) as connection :
        cursor = connection.cursor()
        query = f"SELECT * FROM CUSTOMER_PRODUCT_SUMMARY WHERE PRODUCT_CODE LIKE '%{product_name}%' AND QUOTE_DATE LIKE '%{quote_date}%'"
        quote_source = pd.read_sql_query(query, connection)
        query = f"SELECT * FROM ORDER_SUMMARY WHERE PRODUCT_CODE LIKE '%{product_name}%'"
        order_source = pd.read_sql_query(query, connection)
    return quote_source.sort_values(by = ["QUOTE_DATE"], ascending=False), order_source.sort_values(by = ["ORDER_DATE"], ascending=False)

a, b = FIND_QUOTE("89MZ", "2024")

In [85]:
a

,ID,PRODUCT_CODE,DRAWING_NUMBER,SIZE,RECESS_PARTIAL_THREAD,MOQ,QUANTITY,WEIGHT,M_BOX,KG_BOX,...,TOTAL_PRICE_M,DESCRIPTION,ORDER_ADDRESS,QUOTE_DATE,ORDER_TYPE,AGENT,CUSTOMER_CODE,ORDER_NUMBER,SERIAL_NUMBER,NOTE
1,15661,89MZ,D012DW005,8x1-1/4,-,137,4320.0,11059.2,5.0,12.8,...,4.67,"品名： 乾牆螺絲 十字#2 凸鈕扣頭, 雙牙單出(SS), 滲碳S.H, 鍍鋅三價(不得超過...",Z:/業務部/業務一課/G-報價/1. 外銷/D01200 - Intercorp Stro...,20240321,E,RURU,D01200,Hiro議價,1.712052e+09,None
0,7781,89MZ,D012DW005,8x1-1/4,-,137,1200.0,3072.0,5.0,12.8,...,4.71,"品名： 乾牆螺絲 十字#2 凸鈕扣頭, 雙牙單出(SS), 滲碳S.H, 鍍鋅三價(不得超過...",Z:/業務部/業務一課/G-報價/1. 外銷/D01200 - Intercorp Stro...,20240314,E,RURU,D01200,Hiro,1.710496e+09,None


In [78]:
b.iloc[0,:]

ORDER_ADDRESS                                                         69
S_C                                                             24030027
ITEM                                                                  14
SPEC                                                           #8X1-1/4"
PRODUCT_CODE                                                       #89MZ
UNIT                                                                   M
ORDER_QUANTITY                                                 3,360.000
KG_Mpcs                                                             2.56
ORDER_NUMBER                                                      113457
CUSTOMER_CODE                                                     D01200
ORDER_DATE                                                     2024/3/25
QUANTITY_PER_CARTON                                                  5.0
QUANTITY_PER_BOX                                                     0.0
MERCHANDISE            1022A, 乾牆螺絲 十字#2 凸鈕扣頭, 雙牙單出(

In [ ]:
b[""]